In [1]:
# this code is to generate backtest data for the Q function
import dynamic_Q as Q
import requests
import numpy as np
from scipy.linalg import expm
from skopt import BayesSearchCV
from skopt.space import Integer
from datetime import datetime, timedelta
import pandas as pd

def lobster_data(data1, data2):
    # Extract time from the first column of data1
    Time = data1[:, 1]

   # Assuming data2 is a NumPy array
    num_rows, num_cols = data2.shape
    num_levels = num_cols // 4

    # Reshape data2 to match the desired structure
    data_reshaped = data2.reshape((num_rows, num_levels, 4))

    # Extract MOBid, bid_volume, MOAsk, and ask_volume from reshaped data
    MOAsk = data_reshaped[:, :, 0]  # Ask Price
    ask_volume = data_reshaped[:, :, 1]  # Ask Size 
    MOBid = data_reshaped[:, :, 2]  # Bid Price
    bid_volume = data_reshaped[:, :, 3]  # Bid Size

    return MOBid, bid_volume, MOAsk, ask_volume, Time
# Replace 'your_file.csv' with the actual path to your CSV file
file_path1 = 'C:\\Users\\Maxparizot\\Documents\\GitHub\\TradeonQ\\LOB3Example - Matlab example\\LOBSupportingFiles\\MSFT_2012-06-21_34200000_57600000_message_10.csv'
file_path2 = 'C:\\Users\\Maxparizot\\Documents\\GitHub\\TradeonQ\\LOB3Example - Matlab example\\LOBSupportingFiles\\MSFT_2012-06-21_34200000_57600000_orderbook_10.csv'

# Read the CSV file into a DataFrame
data1 = np.loadtxt(file_path1,delimiter=',')
data2= np.loadtxt(file_path2,delimiter=',')

MOBid, bid_volume, MOAsk, ask_volume, Time = lobster_data(data1, data2)

# Calculate midprice for each entry in MoBid and MoAsk
S = np.array([Q.midprice(MOBid[price, 0], MOAsk[price, 0]) for price in range(MOBid.shape[0])])

# find Imbalance index
I = np.array([Q.imbalance_ratio(bid_volume[time, :], ask_volume[time, :]) for time in range(bid_volume.shape[0])])


# Set your reference time (midnight)
reference_time = datetime(2012, 6, 21, 0, 0, 0)
t = [reference_time + timedelta(seconds=duration) for duration in Time]

# Creating a DataFrame in Python using Pandas
Data = pd.DataFrame({'t': t, 'S': S, 'I': I, 'MOBid': MOBid[:,0], 'MOAsk': MOAsk[:,0]})
# 't' is the timestamp, set it as the index

#separate data into training and validating data
bp = round(0.80 * len(t))  # Use 80% of data for training
TData = Data.iloc[:bp, :]  # Training data
VData = Data.iloc[bp:, :]  # Validation data

In [2]:
import multiprocessing as mp

#this code block is to dynamically build Q until convergence
#it can be put into a function and all values of interest can be returned. However! You want to store everything to see the result at it progresses.
import dynamic_Q as Q
import requests
import numpy as np
from scipy.linalg import expm
from skopt import BayesSearchCV
from skopt.space import Integer
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from scipy.signal import convolve
import time

def calculate_Q(queue, Data, n, N):
    Qt= Q.make_Q(Data, n, N)
    queue.put(Qt)
    return

# Create a Queue object
queue = mp.SimpleQueue()
process = None

# Initialize Variables
Data = pd.DataFrame({'t': [], 'S': [], 'I': [], 'MOBid':[], 'MOAsk': []})
inhomogeneity_convergence = []
Qt_old = np.array([])
Qt = np.array([np.nan])

#initialize parameters
n=5
N=20
counter = 0
tolerance = 0.1 #Convergence Tolerance

print('starting while loop')
while True:
    start_time = time.time()
    ask_prices, ask_volumes, ask_timestamps, bid_prices, bid_volumes, bid_timestamps, mid_price,ImbalanceRatio = Q.get_LOB(requests.get('https://api.kraken.com/0/public/Depth?pair=XBTUSD'))
    #print('got data', counter)
    
    # Create a new row
    new_row = {'t': bid_timestamps[0], 'S': mid_price, 'I': ImbalanceRatio, 'MOBid':bid_prices[0], 'MOAsk': ask_prices[0]}
    # Append the new row to the DataFrame
    Data=pd.concat([Data, pd.DataFrame(new_row, index=[0])], ignore_index=True)

    # Increment the counter
    counter += 1

    # Every 25 iterations, run the Q function
    if counter % 25==0:
        # If a previous process finished running, get its result
        if process and process.is_alive() == False:
            print('getting Q from previous process', counter)
            try:
                Qt = queue.get(block=False)
            except Exception:
                print("Queue is empty")
            process = None

        # Start a new process to calculate Qt if there is no process running
        if process is None:
            print('running Q on separate process', counter)
            process = mp.Process(target=calculate_Q, args=(queue, Data, n, N))
            process.start()
            
        if not np.isnan(Qt).any():
            print("Qt no longer has NaN values.")
            Q_diff=Qt_old - Qt
            Inhomogeneity= sum ((Qt_old > 0.5) & (Qt < 0.5) | (Qt_old < 0.5) & (Qt > 0.5)) #trading inefficiencies, which result from indices (market states) where one matrix gives a trading cue (probability value > 0.5) and the other does not.
            inhomogeneity_convergence.append(Inhomogeneity) #store data for future plotting

            if max(Q_diff) < tolerance:
                print("Q has converged in {} values.".format(counter))
                break
            Qt_old = Qt
    
    end_time = time.time()
    if end_time - start_time <.33 :
        time.sleep(.33 - (end_time - start_time))
    


print("Number of cpu : ", mp.multiprocessing.cpu_count())

starting while loop
running Q on separate process 25
getting Q from previous process 50
Queue is empty
running Q on separate process 50


KeyboardInterrupt: 